<a href="https://colab.research.google.com/github/AndrewZou/Coursera_Capstone/blob/master/Segmenting_Clustering_GEOCODES_Neighbohood_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Geocoding library 
!pip install geopy
!pip install folium

In [0]:
from geopy.geocoders import Nominatim
import folium

In [0]:
# @hidden_cell
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")

In [7]:
from google.colab import files
uploaded = files.upload()

Saving toronto_postcode_brough_neighborhood.CSV to toronto_postcode_brough_neighborhood.CSV


In [35]:
import io
import pandas as pd

#df = pd.read_csv(io.StringIO(uploaded["toronto_postcode_brough_neighborhood.CSV"].decode('utf-8')))
df = pd.read_csv(r"toronto_postcode_brough_neighborhood.CSV")
df.head()
#print(df)

,Unnamed: 0,Postcode,Borough,Neighbourhood
0,0,M1B,Scarborough,"Rouge, Malvern"
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


In [0]:
# A code cell to list all available combination values of Neighbourhood, Brough, Toronto, Ontario as GECOCODE API call input
geo_codes = [', '.join([df['Neighbourhood'][i].split(',')[0], df['Borough'][i], 'Toronto, Ontario']) for i in range(len(df))]

#geo_codes
#for i in range(len(geo_codes)):
#  print(geo_codes[i])

#### Modify the GEOCODES string which get None values thru the API call

In [37]:
# A test cell to get the GEOCODE for a particulor Brough. It works
#Ontario, M8W 3T7, Canada
#Golden Horseshoe, Ontario, Canada - work
#Canada Post Gateway Processing Centre, Mississauga, Toronto, Ontario
#code_test = geolocator.geocode('Mississauga, Toronto, Ontario')
for i in range(len(geo_codes)):
  code_test = geolocator.geocode(geo_codes[i])
  if code_test is None:
    print(geo_codes[i])
    str_array = geo_codes[i].split(',')
    geo_codes[i] = geo_codes[i].replace(str_array[0], '').strip(',').strip()
    print(geo_codes[i])
  

CFB Toronto, North York, Toronto, Ontario
North York, Toronto, Ontario
Studio District, East Toronto, Toronto, Ontario
East Toronto, Toronto, Ontario
Cabbagetown, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Harbourfront, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
St. James Town, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Berczy Park, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Adelaide, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Harbourfront East, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Design Exchange, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Commerce Court, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Harbord, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
Chinatown, Downtown Toronto, Toronto, Ontario
Downtown Toronto, Toronto, Ontario
CN Tower, Do

In [38]:
# Create a new DataFrame for loop through the geocodes with fixed index sequence
df_geocodes = pd.DataFrame(geo_codes, columns = ['GEO_CODE_STR'])
df_geocodes.head()

,GEO_CODE_STR
0,"Rouge, Scarborough, Toronto, Ontario"
1,"Highland Creek, Scarborough, Toronto, Ontario"
2,"Guildwood, Scarborough, Toronto, Ontario"
3,"Woburn, Scarborough, Toronto, Ontario"
4,"Cedarbrae, Scarborough, Toronto, Ontario"


In [39]:
# Because We just concerned to have the GEO code for Postcodes only. The value of Borough is good enough to achieve this.
#geocodes = [geolocator.geocode(', '.join([df['Borough'][i], 'Toronto, Ontario'])) for i in range(len(df))]

geocodes = [geolocator.geocode(df_geocodes['GEO_CODE_STR'][i]) for i in range(len(df_geocodes))]
geocodes

[Location(Rouge, Scarborough—Rouge Park, Scarborough, Toronto, Ontario, M1B 2K5, Canada, (43.8049304, -79.1658374, 0.0)),
 Location(Highland Creek, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1C 3T7, Canada, (43.7901172, -79.1733344, 0.0)),
 Location(Guildwood, 1, Westlake Road, Guildwood Village, Scarborough—Guildwood, Scarborough, Toronto, Ontario, M1E 1L8, Canada, (43.7548985, -79.1977755, 0.0)),
 Location(Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada, (43.7598243, -79.2252908, 0.0)),
 Location(Cedarbrae Collegiate Institute, 550, Markham Road, Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada, (43.75646655, -79.226692442588, 0.0)),
 Location(Scarborough Village, Scarborough—Guildwood, Scarborough, Toronto, Ontario, M1J 3M5, Canada, (43.7437422, -79.2116324, 0.0)),
 Location(Birchmount Park, Scarborough Southwest, Scarborough, Toronto, Ontario, M1K 0A4, Canad

In [11]:
#geocodes = [geolocator.geocode(', '.join([df['Neighbourhood'][i].split(',')[0], df['Borough'][i], 'Toronto, Ontario'])) for i in range(len(df))]
                               
#geocodes

#The above code is not completly work. Some of the value of Neighbourhood has trouble to get geocode.
print("This code cell is not working as we expected. Some of the GEOCODE will be NONE.")

This code cell is not working as we expected. Some of the GEOCODE will be NONE.


In [40]:
# Add geocode to the dataframe
df['latitude'] = [g.latitude for g in geocodes]
df['longitude'] = [g.longitude for g in geocodes]
df.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,latitude,longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.804930,-79.165837
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.754899,-79.197776
3,3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692


In [53]:
# Drop the extra column
df.drop(columns=['Unnamed: 0'], axis = 1)
#df.columns.values
#df.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge, Malvern",43.804930,-79.165837
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.754899,-79.197776
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692
5,M1J,Scarborough,Scarborough Village,43.743742,-79.211632
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.714167,-79.271109
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.708823,-79.295986
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.721939,-79.236232
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.702112,-79.260091
